## Imports

In [0]:
import sys
import os
sys.path.append(os.path.abspath('../utils/'))

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType
import json
from dwh_utils import *
from transformation_utils import *

## Assumption
- We have enabled CDC on the bronze table
- So in production, i would use readstream and handle CDC records through foreachbatch

In [0]:
df = spark.read.table("acme_ecommerce.bronze.products")

- I have to let "product_name", "state" go to ensure there are no duplicates. I had plan to keep all production dimensions and join with orders based on product_id and unit_price. `price_per_product` contains some malformed values so I can not do it and it leads me to dropping duplicated records so we can potentially loose some dimension in the products table. 

## Transformations

In [0]:
df = deduplicate_data_by_time(spark=spark, df=df, partition_cols=["product_id"], order_col="ingest_timestamp")

## Creating delta table
- first run is insert and then upsert

In [0]:
write_to_delta_with_cdc_by_name(
    spark=spark, df=df, catalog="acme_ecommerce", schema="silver", table="products", merge_keys=["product_id"], append_only=False
)